In [10]:
import pandas as pd
import string

In [3]:
!ls ./recipeVectors

allRecipes_recipes_appetizers_salads.json  recipeVectors  test.py
recipes					   scrapy.cfg


In [4]:
df = pd.read_json('./recipeVectors/allRecipes_recipes_appetizers_salads.json', dtype={})
print len(df)
df

8288


,cookingTime,ingredients,instructionSteps,name,rating,ratingCount,url
0,PT20M,"[1 (15 ounce) can black beans, rinsed and drai...",[Place black beans in a medium size mixing bow...,Tex Mex Black Bean Dip,4.59,99,http://allrecipes.com/recipe/14848/tex-mex-bla...
1,PT15M,"[24 slices French baguette, 1 tablespoon butte...",[Preheat your oven's broiler. Spread a thin la...,Strawberry Bruschetta,4.51,33,http://allrecipes.com/recipe/69225/strawberry-...
2,PT30M,"[3 cups diced cooked rotisserie chicken, 2 (8 ...",[Preheat oven to 400 degrees F (200 degrees C)...,Baked Buffalo Chicken Dip,4.74,163,http://allrecipes.com/recipe/221131/baked-buff...
3,PT40M,"[2 tablespoons butter, 2 tablespoons minced gr...",[Preheat oven to 400 degrees F (200 degrees C)...,Perfect Crab-Stuffed Mushrooms,4.55,25,http://allrecipes.com/recipe/229826/perfect-cr...
4,PT40M,"[cooking spray, 24 (1 inch) cubes fresh pineap...",[Preheat oven to 375 degrees F (190 degrees C)...,Easy Rumaki with Pineapple,4.46,19,http://allrecipes.com/recipe/235095/easy-rumak...
5,PT30M,"[1 cup butter, softened, 3 tablespoons poppy s...",[Preheat oven to 350 degrees F (175 degrees C)...,Easy Ham and Cheese Appetizer Sandwiches,4.68,295,http://allrecipes.com/recipe/26780/easy-ham-an...
6,PT35M,"[6 canned artichoke hearts, drained and halved...",[Place artichoke heart halves on a paper towel...,Artichoke Hearts Gratin,4.02,29,http://allrecipes.com/recipe/229454/artichoke-...
7,PT20M,"[1 sweet onion, cut into quarters, 8 cloves ga...",[Blend onion and garlic in a food processor un...,Amazing No Cook Spinach Artichoke Dip,4.45,9,http://allrecipes.com/recipe/223012/amazing-no...
8,PT45M,"[2 kiwis, peeled and diced, 2 Golden Delicious...","[In a large bowl, thoroughly mix kiwis, Golden...",Annie's Fruit Salsa and Cinnamon Chips,4.78,3237,http://allrecipes.com/recipe/26692/annies-frui...
9,PT45M,"[2 (10 ounce) cans chunk chicken, drained, 2 (...",[Heat chicken and hot sauce in a skillet over ...,Buffalo Chicken Dip,4.70,3342,http://allrecipes.com/recipe/68461/buffalo-chi...


In [9]:
pd.set_option('display.max_colwidth', -1)
df.iloc[[10]]['ingredients']

10    [3 avocados - peeled, pitted, and mashed, 1 lime, juiced, 1 teaspoon salt, 1/2 cup diced onion, 3 tablespoons chopped fresh cilantro, 2 roma (plum) tomatoes, diced, 1 teaspoon minced garlic, 1 pinch ground cayenne pepper (optional), Add all ingredients to list, Add all ingredients to list]
Name: ingredients, dtype: object

In [18]:
df[['ingredients']].values[10][0]

[u'3 avocados - peeled, pitted, and mashed',
 u'1 lime, juiced',
 u'1 teaspoon salt',
 u'1/2 cup diced onion',
 u'3 tablespoons chopped fresh cilantro',
 u'2 roma (plum) tomatoes, diced',
 u'1 teaspoon minced garlic',
 u'1 pinch ground cayenne pepper (optional)',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

In [68]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        translate_table = dict((ord(char), unicode(translate_to))
                               for char in not_letters)
    else:
        assert isinstance(to_translate, str)
        translate_table = string.maketrans(not_letters,
                                           translate_to
                                              *len(not_letters))
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        'bitesize',
        'taste',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
    ]
    
    preparatory_descriptions = [
        'blended',
        'boiled',
        'broken',
        'browned',
        'chilled',
        'chopped',
        'cooked',
        'cooled',
        'cored',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'diced',
        'drained',
        'dried',
        'filtered',
        'fresh',
        'frozen',
        'grated',
        'ground',
        'halved',
        'heated',
        'juiced',
        'mashed',
        'minced',
        'mixed',
        'opened',
        'peeled',
        'pitted',
        'prepared',
        'pressed',
        'quartered',
        'raw',
        'rinsed',
        'ripe',
        'ripened',
        'seeded',
        'separated',
        'shaved',
        'shredded',
        'sifted',
        'smashed',
        'sliced',
        'slivered',
        'soaked',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'unopened',
        'unpeeled',
        'warmed',
    ]
    
    other_words_to_filter = [
        'add',
        'all',
        'and',
        'as',
        'in',
        'into',
        'ingredients',
        'list',
        'needed',
        'optional',
        'or',
        'other',
        'to',
        'thin', 'thinly',
        'thick', 'thickly',
        'fine', 'finely',
        'coarse', 'coarsely',
        'large',
        'medium',
        'small',
    ]
    
    print "Before..."
    print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    #print cleaned_ingred_list
    
    # split on whitespace
    # throw out meas units
    print
    print "After..."
    ingreds_only = [' '.join(w for w in ingred_item.split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
         for ingred_item in cleaned_ingred_list
         ]
    print ingreds_only
    
    return

extract_ingredients(df[['ingredients']].values[111][0])

Before...
[u'1 large russet potato, peeled and quartered', u'1 large sweet potato, peeled and quartered', u'1 cup corn', u'1 teaspoon prepared Dijon-style mustard', u'2 tablespoons fresh lime juice', u'3 tablespoons chopped fresh cilantro', u'1 clove garlic, minced', u'3 tablespoons canola oil', u'1/2 teaspoon salt', u'1/4 teaspoon ground black pepper', u'1 cucumber, halved lengthwise and chopped', u'1/2 red onion, thinly sliced', u'1/4 cup finely chopped peanuts', u'Add all ingredients to list', u'Add all ingredients to list']

After...
[u'russet potato', u'sweet potato', u'corn', u'dijonstyle mustard', u'lime juice', u'cilantro', u'garlic', u'canola oil', u'salt', u'black pepper', u'cucumber lengthwise', u'red onion', u'peanuts', '', '']


In [29]:
import re

def convertCookingTimeToMinutes(text):
    text = text.replace('PT','')
    arr = re.findall(r'[0-9]+', text)
    minute = 0
    if len(arr) == 2:
        minute = int(arr[0])* 60 + int(arr[1])
    if len(arr) == 1:
        if 'M' in text:
            minute = int(arr[0])
        if 'H' in text:
            minute = int(arr[0]) * 60
    return minute
df['cookingTimeMinutes'] = df['cookingTime'].apply(convertCookingTimeToMinutes)
df[['cookingTime','cookingTimeMinutes']]

,cookingTime,cookingTimeMinutes
0,PT15M,15
1,PT10M,10
2,PT9H,540
3,PT35M,35
4,PT2H20M,140
5,PT2H30M,150
6,PT35M,35
7,PT55M,55
8,PT1H,60
9,PT2H31M,151
